# Find Conditions for Average Binding Fraction

The linker length distribution is highly dependent on the binding state of the system. Meanwhile, the binding state of the system depends on the linker length distribution. To control the binding fraction while simulating linker lengths, we will iterate between sliding and binding simulations.

#### Approach

First, we will assume that all pairs of adjacent nucleosomes are within the interaction radius, meaning the value of gamma is `1` for all nucleosomes. We will generate a theoretical isotherm for this system, and we will identify the chemical potential that produces approximately 50% (or some other setpoint) of sites bound by our reader. We will then simulate nucleosome sliding to generate a linker length distribution for this chemical potential. We will update the values of gamma based on the sliding simulation and we will reiterate. We will repeat the iteration process until the chemical potential at 50% binding and the associated linker length distribution converge.

#### Specify Package Root Directory

In [1]:
import os
import sys

# Get the absolute path of the notebook's directory
notebook_directory = os.path.dirname(os.path.abspath('__file__'))

# Navigate one level up to get the root directory path
root_directory = os.path.abspath(os.path.join(notebook_directory, '..'))

# Change the working directory to the root directory
os.chdir(root_directory)

# Append the root directory to sys.path
sys.path.append(root_directory)

#### Import Modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sliding_nucleosome.nucleo_arr as nuc
from sliding_nucleosome import mc
from binding_model.theory_bind import find_mu_for_binding_frac

#### Initialize Physical Parameters

In [3]:
# Specify physical parameters
J = np.atleast_2d([-3.92])
B = np.atleast_2d([-1.5])
mu = np.array([-9.7])

In [4]:
# Generate a methylation sequence
n_beads = 500
frac_methyl = 0.1
methyl_corr_length = 18.4
marks = nuc.gen_meth(n_beads, frac_methyl, methyl_corr_length)
marks = np.atleast_2d(marks).T

In [5]:
# Let's force the mark profile to match the one that should produce phase separation
marks = np.array(
    [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 2., 1., 1., 1., 1., 1., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 2., 1., 1., 1., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
   1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 2., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   0., 0., 0., 0., 0., 0., 0.]
)
marks = np.atleast_2d(marks).T

In [6]:
# Specify the polymer
gamma = np.ones(marks.shape[0])
nbi = np.array([2])
linker_corr_length = 45
linker_lengths = np.random.exponential(linker_corr_length, size=marks.shape[0])
linker_lengths = np.maximum(linker_lengths, 1.0)
linker_lengths = linker_lengths.astype(int)
a = int(np.floor(15.1))
lam = -np.log(1 - 1 / linker_corr_length)

In [7]:
print(f"Linker Lengths: \n\n{linker_lengths}")

Linker Lengths: 

[ 28   1  32  10  41   3  66 130  21  21  27  24 131  96  39  49   1  55
  76 102  91 139  35  85  15   1  15  38   6 159  11 168  20   5  94  26
 202  14 125  54  44  59 104  26   5  25  18   6  47 109  45   1 103 161
 113  62  85  60 104  47  96  55  34   9 138  59  10  60  10 164  94  37
  95 104  26   7  74   2  16  45  10  36 121 132  49  12  45  83 121  32
  76   8  84  79  20  25  10  27  23  31   1   3   6 136   8  20  22   5
  14  18  34  81  38  82  26   8  15  26  50  49  57  17   3   9  35  40
  35   1  14  15   4   3  46  42  22 135  24   8   9 148 175   1 107  62
  16 171  18  48  46  16  24  53  46  46   5  22  44  18  66  59  41  59
  25 105  51  21   3   8  11  31  21  32 181 154  52  13  62  74 122  54
   6  27  73  11  10  27  66  34  65 165   7  30  49 153  27  91  47 227
  56 198  23   2   9  19   9  29  43  19   4  29  78  28   4  25   5  10
 110  46  26  41  18  13  17  43 110  98  37  53  44 162  59   3  50  35
 104  37   1  21   6  12  41   1 

#### Initialize the Nucleosome Array

In [8]:
nuc_arr = nuc.NucleosomeArray(
    J = J,
    B = B,
    mu = mu,
    linker_lengths = linker_lengths,
    a = a,
    lam = lam,
    marks = marks,
    Nbi = nbi
)

In [9]:
print(f"Average Gamma: {np.average(nuc_arr.gamma)}")

Average Gamma: 0.27


#### Specify Parameters for Linker Simulation

In [10]:
n_snap = 1
n_steps_per_snap = 1000

#### Iterate Between Binding and Linker Simulations

Iterate in the following manner:

1. Run the binding model to select the chemical potential that produces the desired binding fraction for a fixed linker length distribution.

2. Using the chemical potential selected in step 1, update the linker length distribution using the linker model.

3. Iterate between steps 1 and 2 until the chemical potential and average `gamma` parameter converge.

In [11]:
target_binding_fraction = 0.5
mu_lower = -20
mu_upper = 0

nuc_arr = mc.find_nuc_arr_for_avg_binding(
    nuc_arr,
    mu_lower,
    mu_upper,
    target_binding_fraction,
    n_snap,
    n_steps_per_snap
)

Starting bind/slide iteration 1 of 10...
Binding Converged!
Running linker simulation...
Mu: -3.59375, Avg. Gamma: 1.0
Reiterating...

Starting bind/slide iteration 2 of 10...
Binding Converged!
Running linker simulation...
Mu: -9.453125, Avg. Gamma: 1.0
Reiterating...

Starting bind/slide iteration 3 of 10...
Binding Converged!
Running linker simulation...
Mu: -8.28125, Avg. Gamma: 1.0
Reiterating...

Starting bind/slide iteration 4 of 10...
Binding Converged!
Running linker simulation...
Mu: -9.609375, Avg. Gamma: 1.0
Reiterating...

Starting bind/slide iteration 5 of 10...
Binding Converged!
Running linker simulation...
Mu: -6.5625, Avg. Gamma: 1.0
Reiterating...

Starting bind/slide iteration 6 of 10...
Binding model iteration: 10
Binding Converged!
Running linker simulation...
Mu: -9.365234375, Avg. Gamma: 1.0
Reiterating...

Starting bind/slide iteration 7 of 10...
Binding Converged!
Running linker simulation...
Mu: -8.59375, Avg. Gamma: 1.0
Reiterating...

Starting bind/slide it